In [1]:
import sys
sys.path.append('../') # Retrocede una carpeta atrás
from config.paths import SRC_DIR
import os
from dotenv import load_dotenv
load_dotenv()

from utils.scraping_utils import ScrapingUtils

In [2]:
url_to_extract_urls = "https://vespa-colombia.com/vespa-gts-300-super-sport/"
brand_to_scrape = "Vespa"

In [3]:
scraping_utils = ScrapingUtils(api_key=os.getenv("FIRECRAWL_API_KEY"))

In [4]:
data_scraped = scraping_utils.get_data_from_website(url_to_extract_urls)

In [102]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(data_scraped.html, "html.parser")
title_element = soup.find(class_="product_title entry-title elementor-heading-title elementor-size-default")
product_title = title_element.get_text(strip=True) if title_element else None
product_title

In [103]:
import re

def extract_image_and_pdf_links(markdown_text):
    """
    Extrae todos los links de imágenes y PDFs que aparecen en el markdown.
    """
    # Extensiones de imagen y PDF a buscar
    image_extensions = ['.webp', '.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff', '.svg']
    pdf_extensions = ['.pdf']
    all_extensions = image_extensions + pdf_extensions

    # Busca los links en formato ![](url) y [nombre](url)
    pattern = r'\((https?://[^\s)]+|/[^)\s]+)\)'
    matches = re.findall(pattern, markdown_text)

    # Filtra links que terminen con alguna de las extensiones buscadas
    links = [link for link in matches if any(link.lower().endswith(ext) for ext in all_extensions)]
    return links

links = extract_image_and_pdf_links(data_scraped.markdown)

In [104]:
imagenes = []
pdfs = []
for item in links:
    image_extensions = [".webp", ".jpg", ".jpeg", ".png", ".gif", ".bmp", ".tiff", ".svg"]
    if any(ext in item.lower() for ext in image_extensions):
        if '300x300' not in item:
            imagenes.append(item)
    elif ".pdf" in item:
        pdfs.append(item)

# Eliminar duplicados
imagenes = list(set(imagenes))
pdfs = list(set(pdfs))

In [105]:
pdfs

['https://loscoches.com/wp-content/uploads/2019/07/politica-de-proteccion-de-datos-personales-v4.pdf',
 'https://vespa-colombia.com/wp-content/uploads/2025/01/BROCHURE-VESPA-GTS-2024.pdf']

In [106]:
imagenes

['https://grupopiaggio.com.co/wp-content/uploads/2021/02/icon-whatsapp-white.svg',
 'https://vespa-colombia.com/wp-content/uploads/2021/03/motorcycle.svg']

In [107]:
import os
import requests

def descargar_archivos(lista_urls, carpeta_destino):
    """
    Descarga archivos desde una lista de URLs y los guarda en la carpeta de destino.
    Si se recibe un error 406, lo intenta de nuevo enviando headers de navegador.
    """
    if not os.path.exists(carpeta_destino):
        os.makedirs(carpeta_destino)

    # Headers tipo navegador para evadir errores como 406
    browser_headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'es-ES,es;q=0.9,en;q=0.8',
        'Connection': 'keep-alive',
        'Referer': 'https://fratelliglobal.com/',
    }

    for url in lista_urls:
        try:
            nombre_archivo = url.split("/")[-1]
            ruta_completa = os.path.join(carpeta_destino, nombre_archivo)

            # Si ya existe, no descargar de nuevo
            if os.path.exists(ruta_completa):
                print(f"Archivo ya existe: {nombre_archivo}")
                continue

            try:
                response = requests.get(url, stream=True, timeout=20)
                response.raise_for_status()
            except requests.exceptions.HTTPError as e:
                if response.status_code == 406:
                    print(f"406 recibido para {url}, reintentando con headers de navegador...")
                    response = requests.get(url, stream=True, timeout=20, headers=browser_headers)
                    response.raise_for_status()
                else:
                    raise

            with open(ruta_completa, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:  # filtra paquetes vacíos
                        f.write(chunk)
            print(f"Descargado: {nombre_archivo}")
        except Exception as e:
            print(f"Error al descargar {url}: {e}")


In [108]:
carpeta_destino = f"{SRC_DIR}/data/scraped_data_downloaded/{product_title}"
descargar_archivos(pdfs, carpeta_destino)
descargar_archivos(imagenes, carpeta_destino)

Descargado: politica-de-proteccion-de-datos-personales-v4.pdf
Descargado: BROCHURE-VESPA-GTS-2024.pdf
Descargado: icon-whatsapp-white.svg
Descargado: motorcycle.svg
